<a href="https://colab.research.google.com/github/Menager49/PY-net-elec/blob/main/Projet_Net_Elec_Equipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet Net Elec

Promotion DA continue - Dec. 2021 
Membres de l'équipe : Christine, Pierre-Emmanuel, Cédric
Mentor : Pierre

# Partie 1 : Chargement et Preprocessing <a class="anchor" id="Partie01"><a>

## 1.1 Chargements modules et connexion Google Drive

In [ ]:
# Montage du Drive à Colab pour hebergé le dataset

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Chargement des modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

## 1.2 Preprocessing DataSet Energie

### 1.2.1 Chargement

In [ ]:
df_energie = pd.read_csv(filepath_or_buffer='/content/drive/MyDrive/DataScientest_Projet Net Elec/eco2mix-regional-cons-def.csv', 
                 sep=';', 
                 parse_dates = {'Date_Heure': ['Date', 'Heure'] })

In [ ]:
#df_energie.head(5)

### 1.2.2 Filtrage

In [ ]:
#df_energie.info()

In [ ]:
### Liste de varaibles à supprimer
col_to_drop = ['Column 26',
              'Code INSEE région',
              'Date_Heure'
              ]
### Suppression des varaibles inutiles
df_energie = df_energie.drop(col_to_drop, axis=1)

In [ ]:
# df_energie.info()

In [ ]:
# Filtrage temporel

# Conversion du type pour le champs date
df_energie['Date - Heure'] = pd.to_datetime(df_energie['Date - Heure'], utc=True)

# Limites temporelles haute et basse
df_energie = df_energie[(df_energie['Date - Heure'] >= '01/01/2016') & (df_energie['Date - Heure'] <= '31/12/2021')]

### 1.2.3 Standardisation

In [ ]:
# Les champs sont renommés : 
Dic_Noms = {
    'Région'             :'region_nom',
    'Nature'             :'nature',
    'Date - Heure'       :'date',
    'Consommation (MW)'  :'conso_inter',
    'Pompage (MW)'       :'conso_hydro',
    'Thermique (MW)'     :'prod_therm',
    'Nucléaire (MW)'     :'prod_nuc',
    'Solaire (MW)'       :'prod_sol',
    'Hydraulique (MW)'   :'prod_hydro',
    'Bioénergies (MW)'   :'prod_bio',
    'Eolien (MW)'        :'prod_eol',
    'Ech. physiques (MW)':'echanges',
    'TCO Thermique (%)'  :'tco_therm', 
    'TCO Nucléaire (%)'  :'tco_nuc',
    'TCO Eolien (%)'     :'tco_eol', 
    'TCO Solaire (%)'    :'tco_sol', 
    'TCO Hydraulique (%)':'tco_hydro',
    'TCO Bioénergies (%)':'tco_bio', 
    'TCH Nucléaire (%)'  :'tch_nuc', 
    'TCH Eolien (%)'     :'tch_eol',
    'TCH Thermique (%)'  :'tch_therm', 
    'TCH Solaire (%)'    :'tch_sol', 
    'TCH Hydraulique (%)':'tch_hydro', 
    'TCH Bioénergies (%)':'tch_bio', 
            }

df_energie = df_energie.rename(Dic_Noms, axis=1)

### 1.2.4 Complétude (Doublons & NANs)

In [ ]:
# Identification des lignes dupliquées
df_energie.duplicated().sum()

72

In [ ]:
# Traitement des lignes dupliquées
df_energie = df_energie.drop_duplicates()

In [ ]:
# Identification des NANs
df_energie.isna().sum(axis = 0)

region_nom           0
nature               0
date                 0
conso_inter          0
prod_therm           0
prod_nuc        525815
prod_eol             0
prod_sol             0
prod_hydro           0
conso_hydro     525815
prod_bio             0
echanges             0
tco_therm       841464
tch_therm       841464
tco_nuc        1016669
tch_nuc        1016669
tco_eol         841464
tch_eol         841464
tco_sol         841464
tch_sol         841464
tco_hydro       841464
tch_hydro       841464
tco_bio         841464
tch_bio         841464
dtype: int64

In [ ]:
# Traitement des NANs:

# Cas 1 : Suppression des lignes
df_energie.dropna(subset = ['conso_inter'], axis=0, inplace = True)

# Cas 2 : Remplacement par zéro
list_fillzero=['prod_nuc', 'prod_therm', 'prod_eol', 'prod_sol', 'prod_hydro', 'prod_bio', 'conso_hydro']

for i in list_fillzero:
    df_energie[i] = df_energie[i].fillna(0)

# Cas 3 : Remplacement par zéro
list_fillzero=['tco_therm', 'tch_therm', 'tco_nuc', 'tch_nuc', 'tco_eol', 'tch_eol', 'tco_sol','tch_sol','tco_hydro','tch_hydro','tco_bio','tch_bio']

for i in list_fillzero:
    df_energie[i] = df_energie[i].fillna(0)
    

In [ ]:
df_energie.isna().sum(axis = 0)

region_nom     0
nature         0
date           0
conso_inter    0
prod_therm     0
prod_nuc       0
prod_eol       0
prod_sol       0
prod_hydro     0
conso_hydro    0
prod_bio       0
echanges       0
tco_therm      0
tch_therm      0
tco_nuc        0
tch_nuc        0
tco_eol        0
tch_eol        0
tco_sol        0
tch_sol        0
tco_hydro      0
tch_hydro      0
tco_bio        0
tch_bio        0
dtype: int64

### 1.2.5 Standardisation temporelle avec le DataSet Météo

Probleme : pour grouper les TCO et TCH il faut les convertir de % en MW

In [ ]:
# Groupby : Afin d'avoir la même granularité que la variable temps du dataset météo.
# Nous perdons de l'information, mais je ne sais pas comment ré-echantillonner le DF météo afin de dedoubler les lignes.

df_energie['date_index'] = df_energie['date']
df_energie = df_energie.set_index('date_index')

df_energie = df_energie.groupby( ['region_nom','nature', pd.Grouper(key = 'date' , freq = 'H')]).agg({
    'conso_inter' : 'sum', 
    'prod_therm'  : 'sum', 
    'prod_nuc'    : 'sum', 
    'prod_eol'    : 'sum', 
    'prod_sol'    : 'sum',
    'prod_hydro'  : 'sum', 
    'conso_hydro' : 'sum', 
    'prod_bio'    : 'sum', 
    'echanges'    : 'sum',
})


df_energie = df_energie.reset_index()

df_energie = df_energie.sort_values(by = ['date'], ascending = True)

## 1.3 Preprocessing DataSet Meteo

### 1.3.1 Chargement

In [ ]:
df_meteo = pd.read_csv(filepath_or_buffer='/content/drive/MyDrive/DataScientest_Projet Net Elec/donnees-synop-essentielles-omm.csv', 
                 sep=';')

In [ ]:
#df_meteo.head(5)

### 1.3.2 Filtrage

In [ ]:
# Suppression des colonnes inutiles
# Dans un premier temps on ne conserve que la temperature pour simplifier le dataset

List_cols = {'ID OMM station', 
             'Pression au niveau mer',
             'Variation de pression en 3 heures', 
             'Type de tendance barométrique',
             'Direction du vent moyen 10 mn', 
             'Point de rosée', 
             'Humidité', 
             'Visibilité horizontale',
             'Temps présent', 
             'Temps passé 1', 
             'Temps passé 2', 
             "Nébulosité  des nuages de l' étage inférieur",
             "Hauteur de la base des nuages de l'étage inférieur",
             "Type des nuages de l'étage inférieur",
             "Type des nuages de l'étage moyen",
             "Type des nuages de l'étage supérieur", 
             'Pression station',
             'Niveau barométrique', 
             'Géopotentiel',
             'Variation de pression en 24 heures',
             'Température minimale sur 12 heures',
             'Température minimale sur 24 heures',
             'Température maximale sur 12 heures',
             'Température maximale sur 24 heures',
             'Température minimale du sol sur 12 heures',
             'Méthode de mesure Température du thermomètre mouillé',
             'Température du thermomètre mouillé',
             'Rafale sur les 10 dernières minutes', 
             'Rafales sur une période',
             'Periode de mesure de la rafale', 
             'Etat du sol',
             'Hauteur totale de la couche de neige, glace, autre au sol',
             'Hauteur de la neige fraîche', 
             'Periode de mesure de la neige fraiche',
             'Précipitations dans la dernière heure',
             'Précipitations dans les 3 dernières heures',
             'Précipitations dans les 6 dernières heures',
             'Précipitations dans les 12 dernières heures',
             'Précipitations dans les 24 dernières heures', 
             'Phénomène spécial 1',
             'Phénomène spécial 2', 
             'Phénomène spécial 3', 
             'Phénomène spécial 4',
             'Nébulosité couche nuageuse 1', 
             'Type nuage 1', 
             'Hauteur de base 1',
             'Nébulosité couche nuageuse 2', 
             'Type nuage 2', 
             'Hauteur de base 2',
             'Nébulosité couche nuageuse 3', 
             'Type nuage 3', 
             'Hauteur de base 3',
             'Nébulosité couche nuageuse 4', 
             'Type nuage 4', 
             'Hauteur de base 4',
             'Coordonnees', 
             'Nom', 
             'Type de tendance barométrique.1',
             'Temps passé 1.1', 
             'Temps présent.1', 
             'Température (°C)',
             'Température minimale sur 12 heures (°C)',
             'Température minimale sur 24 heures (°C)',
             'Température maximale sur 12 heures (°C)',
             'Température maximale sur 24 heures (°C)',
             'Température minimale du sol sur 12 heures (en °C)', 'Latitude',
             'Longitude', 
             'Altitude', 
             'communes (name)', 
             'communes (code)',
             'EPCI (name)', 
             'EPCI (code)', 
             'department (name)', 
             'department (code)',
             'region (code)',
             'mois_de_l_annee'
            }

df_meteo = df_meteo.drop(List_cols, axis=1)

In [ ]:
# Identification des région à supprimer
# df_meteo['region (name)'].unique()

In [ ]:
# Suppression des région qui sont en dehors du dataset energie
region_to_drop = ['Martinique',
                  'Guyane',
                  'La Réunion',
                  'Guadeloupe',
                  'Mayotte',
                  'Terres australes et antarctiques françaises',
                  'Saint-Pierre-et-Miquelon',
                  'Saint-Barthélemy',
                   'Corse'
                 ]

for j in region_to_drop:
    df_meteo = df_meteo[df_meteo['region (name)'] != j]

In [ ]:
# Contrôle
# df_meteo['region (name)'].unique()

### 1.3.3 Standardisation

In [ ]:
# Les variables sont renommées pour plus d'homogénéité avec le 1er DataFrame

Dic_Noms_meteo = {'Date'             : 'date',
                 'Température'       : 'temperature',
                 'region (name)'     : 'region_nom', 
                 'Vitesse du vent moyen 10 mn': 'vitesse_vent_10min',
                 'Nebulosité totale' : 'nebulosite_totale'
                }
    
df_meteo = df_meteo.rename(Dic_Noms_meteo, axis=1)
df_meteo.columns

Index(['date', 'vitesse_vent_10min', 'temperature', 'nebulosite_totale',
       'region_nom'],
      dtype='object')

In [ ]:
# Temperature : conversion degrès Kevin => degrès Celsius
df_meteo['temperature'] = df_meteo['temperature'] - 273.15

In [ ]:
# date : Conversion du type de variable temps
df_meteo['date'] = pd.to_datetime(df_meteo['date'], utc=True)

In [ ]:
# Standardisation des noms des régions
dico_df_meteo = {'Occitanie'           :'Occitanie',
                 'Normandie'           :'Normandie', 
                 'Bretagne'            :'Bretagne',
                 "Provence-Alpes-Côte d'Azur":"Provence-Alpes-Côte d'Azur", 
                 'Hauts-de-France'     :'Hauts-de-France', 
                 'Île-de-France'       :'Île-de-France',
                 'Grand Est'           :'Grand Est', 
                 'Nouvelle-Aquitaine'  :'Nouvelle-Aquitaine',
                 'Bourgogne-Franche-Comté':'Bourgogne-Franche-Comté', 
                 'Pays de la Loire'    :'Centre-Val de Loire',
                 'Auvergne-Rhône-Alpes':'Auvergne-Rhône-Alpes', 
                 'Centre-Val de Loire' :'Centre-Val de Loire'}

df_meteo = df_meteo.rename(dico_df_meteo, axis = 1)

### 1.3.4 Complétude

**Doublons**

In [ ]:
# Recherche de lignes dupliquées
df_meteo.duplicated().sum()

762

In [ ]:
# Traitement des lignes en doublons
df_meteo = df_meteo.drop_duplicates()

**Region NAN**

In [ ]:
# Correction de la région 'nan'
df_meteo[df_meteo['region_nom'].isna()==True].head(5)

,date,vitesse_vent_10min,temperature,nebulosite_totale,region_nom
1,2010-01-05 09:00:00+00:00,NaN,32.3,60.0,NaN
5,2010-01-05 12:00:00+00:00,9.8,1.9,90.0,NaN
10,2010-01-05 15:00:00+00:00,4.1,-1.9,NaN,NaN
17,2010-01-05 21:00:00+00:00,7.2,2.0,100.0,NaN
19,2010-01-06 00:00:00+00:00,5.1,26.6,NaN,NaN


In [ ]:
# suppression de la région NAN
df_meteo.dropna(subset = ['region_nom'], axis=0, inplace = True)

**Autres NANs**

In [ ]:
df_meteo.isna().sum(axis = 0)

date                       0
vitesse_vent_10min      3448
temperature             8077
nebulosite_totale     569446
region_nom                 0
dtype: int64

In [ ]:
# Traitement des NANs des variables quantitatives

df_meteo['temperature']        = df_meteo['temperature'].fillna(method='pad') #Remplacement par la valeur précédente
df_meteo['vitesse_vent_10min'] = df_meteo['vitesse_vent_10min'].fillna(method='pad') #Remplacement par la valeur précédente
df_meteo['nebulosite_totale']  = df_meteo['nebulosite_totale'].fillna(method='pad') #Remplacement par la valeur précédente

#df_meteo['temperature']        = df_meteo['temperature'].fillna(df_meteo['temperature'].mean())
#df_meteo['vitesse_vent_10min'] = df_meteo['vitesse_vent_10min'].fillna(df_meteo['vitesse_vent_10min'].mean())
#df_meteo['nebulosite_totale']  = df_meteo['nebulosite_totale'].fillna(df_meteo['nebulosite_totale'].mean())

In [ ]:
df_meteo.isna().sum(axis = 0)

date                  0
vitesse_vent_10min    0
temperature           0
nebulosite_totale     0
region_nom            0
dtype: int64

## 1.4 Fusion des deux DataSet

### 1.4.1 Merge

In [ ]:
# jointure des deux dataset

df = df_energie.merge(df_meteo, how='left', left_on=['date', 'region_nom'], right_on=['date', 'region_nom'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1115645 entries, 0 to 1115644
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype              
---  ------              --------------    -----              
 0   region_nom          1115645 non-null  object             
 1   nature              1115645 non-null  object             
 2   date                1115645 non-null  datetime64[ns, UTC]
 3   conso_inter         1115645 non-null  float64            
 4   prod_therm          1115645 non-null  float64            
 5   prod_nuc            1115645 non-null  float64            
 6   prod_eol            1115645 non-null  float64            
 7   prod_sol            1115645 non-null  float64            
 8   prod_hydro          1115645 non-null  float64            
 9   conso_hydro         1115645 non-null  float64            
 10  prod_bio            1115645 non-null  float64            
 11  echanges            1115645 non-null  float64            
 12  

### 1.4.2 Complétude

In [ ]:
df.isna().sum(axis = 0)

region_nom                 0
nature                     0
date                       0
conso_inter                0
prod_therm                 0
prod_nuc                   0
prod_eol                   0
prod_sol                   0
prod_hydro                 0
conso_hydro                0
prod_bio                   0
echanges                   0
vitesse_vent_10min    421042
temperature           421042
nebulosite_totale     421042
dtype: int64

# Partie 2 : Exploration et Story-telling <a class="anchor" id="Partie02"><a>

## 2.1 Exploration sur la consommation d'energie

## 2.2 Exploration sur les sources de productions

## 2.3 Relations Consommation & Production

## 2.4 Relations consommation & Météo

# Partie 3 : Problématique et analyse <a class="anchor" id="Partie03"><a>

## 3.1 Définition de la problématique

## 3.2 Construction d'indicateurs

## 3.3 Données pour Dashboard PowerBI

## 3.4 Prédictions

# Partie 4 : Conclusions  <a class="anchor" id="Partie04"><a>

## 4.1 Apports du projet en réponse à la problématique

## 4.2 Limites et axes d'amélioration